In [1]:
import time

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

ph_code = {'dc':'+','fe':'(','hg':')','ba':'-',
           'yz':'1','wx':'2','vu':'3','ts':'4','rq':'5','po':'6','nm':'7','lk':'8','ji':'9','acb':'0'}

In [3]:
#give the url here
url = 'https://www.justdial.com/Hyderabad/Ready-Mix-Concrete-Dealers-in-Bachupally/nct-10993743'

In [4]:
def getData(divs):
    
    table = []

    for div in divs:

        row = []

        content = str(div)
        
        name = div.find('span','lng_cont_name').get_text()
        votes = div.find('span','lng_vote').get_text().replace('Votes','').strip()
        
        rating = None
        if votes != '0':
            rating = div.find('span','green-box').get_text()
        
        address = None
        address_div = div.find('span','cont_fl_addr')
        if address_div != None:
            address = address_div.get_text()

        numbers_all = div.find('p','contact-info')
        number = None
        if numbers_all != None:
            numbers_all = numbers_all.findAll('span','mobilesv')
            number = ''
            for j in numbers_all:
                number += ph_code[j['class'][1].replace('icon-','')]

        verified = False
        
        if 'jd_verified' in content:
            verified = True

        trusted = False
        if 'jd_trusted' in content:
            trusted = True

        page_link = div.find('h2','store-name').find('a')['href']

        row.extend([name,rating,votes,number,address,verified,trusted,page_link])
        table.append(row)
        
    return table

In [74]:
def correct_phone(phone_no):
    if(phone_no):
        
        if(phone_no.split("-")[1][0]== '4'):
            
            return '040' + phone_no.split("-")[2]
        else:
            return phone_no.split("-")[1]
    

In [9]:
def getProducts(page_url):
    
    response = requests.get(page_url,headers=headers)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    content = soup.find('span','showmore').text
    
    content = ', '.join(content.split(' \r\n\t\t\t\t')).replace('\n','').replace('\r','').replace('\t','').strip()
    
    return content

In [6]:
table = []
i = 0

while True:
    i += 1
    
    response = requests.get(url+'/page-'+str(i),headers=headers)
    page = BeautifulSoup(response.text, 'html.parser')
    divs = page.findAll('li','cntanr')
    
    if len(divs) == 0:
        break
    
    table.extend(getData(divs))
    
    if len(divs) < 10:
        break

In [7]:
dfObj = pd.DataFrame(table, columns = ['Name','Rating','Votes','Phone','Address','Verified','Trusted','Url']) 
dfObj

,Name,Rating,Votes,Phone,Address,Verified,Trusted,Url
0,Omega RMC,4.5,15,+(91)-9152182118,"Flat No.6-150, Near Gandimaisamma Chowrasta, B...",True,True,https://www.justdial.com/hyderabad/Omega-RMC-B...
1,Premier Readymix Concrete,4.7,22,+(91)-9152874536,"Plot No 57, Telcom Nagar, Gachibowli, Hyderaba...",False,False,https://www.justdial.com/Hyderabad/Premier-Rea...
2,Rdc Concrete India Pvt Ltd,4.1,16,+(91)-9152886244,"Plot No 8/A, Survey No 334 to 337, Bollaram Ro...",False,False,https://www.justdial.com/Hyderabad/Rdc-Concret...
3,Metro Rmc,4.0,21,+(91)-9152194066,"39-p , 1ST Floor , B R R Residency ,D Pochamp...",True,True,https://www.justdial.com/Hyderabad/Metro-Rmc-N...
4,G T Associates,4.5,22,+(91)-9152861461,"4-55, Kukatpally, Hyderabad - 500072, Near BJP...",False,False,https://www.justdial.com/Hyderabad/G-T-Associa...
...,...,...,...,...,...,...,...,...
401,Sv Readymix Concrete,None,0,+(91)-9885043343,"Sv Rmc, Kavadipalli Abdhullapur Mettu, Ramoji ...",False,False,https://www.justdial.com/hyderabad/Sv-Readymix...
402,Ready mix concrete,None,0,+(91)-9848619939,"Batasingaram, Rangareddy - 501512, Near Rfc",False,False,https://www.justdial.com/hyderabad/Ready-mix-c...
403,Tawakkal Readymix Concrete,None,0,+(91)-7032253676,"Kongara Kalan, Ibrahimpatnam, Rangareddy - 501...",False,False,https://www.justdial.com/hyderabad/Tawakkal-Re...
404,Bellstone Hi Tech Internati..,4.1,69,+(91)-9152863888,"3755 , Ground Floor, Chawri Bazar, Delhi - 110...",False,False,https://www.justdial.com/Delhi/Bellstone-Hi-Te...


In [10]:
startTime = time.time()
dfObj['Products'] = dfObj['Url'].apply(lambda x : getProducts(x))
print(time.time() - startTime)

647.1311049461365


In [75]:
dfObj['phone_number']= dfObj['Phone'].apply(lambda x: correct_phone(x))

In [78]:
final_csv = dfObj.drop(columns=['Phone'],axis=1)

In [80]:
final_csv.shape

(406, 9)

In [79]:
final_csv.to_csv("Readymix_concrete_hyderabad.csv",sep=",",header=True)